In [10]:
import pickle
import numpy as np
import pprint

In [1]:
with open('trained_parameters.pkl', 'rb') as f:
    parameters = pickle.load(f)

with open('sentences_with_eos_and_unk.pkl', 'rb') as f:
    sentences, counter = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'trained_parameters.pkl'

In [20]:
vocab = sorted(counter.keys())
vocab_size = len(vocab)
print('There are %d unique words in our dataset.' % vocab_size)

word_to_ix = { w:i for i,w in enumerate(vocab) }
ix_to_word = { i:w for i,w in enumerate(vocab) }
#pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(ix_to_word)

There are 18890 unique words in our dataset.


In [21]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [49]:
def sample(parameters, word_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- Python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    word_to_ix -- Python dictionary mapping each character to an index.
    seed -- Used for grading purposes. Do not worry about it.

    Returns:
    indices -- A list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    ### START CODE HERE ###
    # Step 1: Create the a zero vector x that can be used as the one-hot vector 
    # Representing the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size,1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a,1))
    
    # Create an empty list of indices. This is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # idx is the index of the one-hot vector x that is set to 1
    # All other positions in x are zero.
    # Initialize idx to -1
    idx = -1
    
    # Loop over time-steps t. At each time-step:
    # Sample a character from a probability distribution 
    # And append its index (`idx`) to the list "indices". 
    # You'll stop if you reach 50 characters 
    # (which should be very unlikely with a well-trained model).
    # Setting the maximum number of characters helps with debugging and prevents infinite loops. 
    counter = 0
    newline_character = word_to_ix['<eos>']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh( np.dot(Wax,x) + np.dot(Waa,a_prev) + b)
        z = np.dot(Wya,a) + by
        y = softmax(z)
        
        # For grading purposes
        np.random.seed(counter + seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        # (see additional hints above)
        idx = np.random.choice( range(vocab_size) , p = np.ravel(y) )

        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input x with one that corresponds to the sampled index `idx`.
        # (see additional hints above)
        x = np.zeros((vocab_size,1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        
    ### END CODE HERE ###

    if (counter == 50):
        indices.append(word_to_ix['<eos>'])
    
    return indices


def get_sample(sample_ix, ix_to_word):
    txt = ' '.join(ix_to_word[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    return txt

In [48]:
seed = np.random.randint(100) 

sampled_indices = sample(parameters, word_to_ix, seed)
last_sample_sentences = get_sample(sampled_indices, ix_to_word)
print(last_sample_sentences.replace('\n', ''))

Aicf make sure that the ballscrew vertical face page off the activated position <eos>
